In [1]:
!pip install wandb

In [2]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=28757ee8c954ef142eaa01a00a53ccd682b4735704c1ec3f437285fe48aa52f0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import wandb
from rouge_score import rouge_scorer
# from dotenv import load_dotenv
import os
from tabulate import tabulate
from nltk.translate.bleu_score import corpus_bleu
import sympy as sp
# load_dotenv()

In [8]:
df = pd.read_csv('/content/RAG_QA_dataset.csv')

# # If 'Unnamed: 0' column still exists, you can drop it
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# 80% -> Training Data, 20% -> Testing Data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 90% -> Training Data, 10% -> Validation Data
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [5]:
df.head()

,Question,Object,Answer,Context
0,what is a yali ?,yali,a yali is a mythical creature found predominan...,Yali is a mythical creature found predominantl...
1,what animals is a yali typically composed of ?,yali,a yali is typically depicted as a composite of...,Yali is a mythical creature found predominantl...
2,what attributes does a yali symbolize ?,yali,"a yali symbolizes attributes like strength, pr...",Yali is a mythical creature found predominantl...
3,why are yalis considered unique ?,yali,yalis are unique because they do not adhere to...,Yali is a mythical creature found predominantl...
4,which cultures have similar mythical creatures...,yali,cultures that have similar mythical creatures ...,It shares similarities with other mythical cre...


In [9]:
tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-finetuned-squadv1")
model = AutoModelForQuestionAnswering.from_pretrained("valhalla/bart-large-finetuned-squadv1")

def calculate_max_length(column_name):
    df[column_name] = df[column_name].astype(str)
    return df[column_name].apply(lambda x: len(tokenizer.tokenize(x))).max()

max_length_question = calculate_max_length('Question')
max_length_object = calculate_max_length('Object')
max_length_context = calculate_max_length('Context')
max_length_answer = calculate_max_length('Answer')

print(f"Maximum token length in the 'Question' column: {max_length_question + max_length_object + max_length_context}")
print(f"Maximum token length in the 'Answer' column: {max_length_answer}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Maximum token length in the 'Question' column: 184
Maximum token length in the 'Answer' column: 62


In [ ]:
all_text = ' '.join(df['Question']) + ' ' + ' '.join(df['Answer']) + ' ' + ' '.join(df['Object'])
words = list(all_text.split())

len(words)

86396

In [ ]:
from collections import Counter

word_counts = Counter(all_text.split())
word_counts

In [ ]:
unique_words = list(word_counts.keys())
unique_words

In [ ]:
print(len(unique_words))

6033


In [ ]:
def compare_words_with_bart_vocab(word_list):
    # Tokenize the words in the list
    tokenized_words = [token for word in word_list for token in tokenizer.tokenize(word)]

    # Get BART's vocabulary
    bart_vocab = set(tokenizer.get_vocab().keys())

    # Check which tokens are in or not in BART's vocabulary
    words_in_vocab = {word for word in tokenized_words if word in bart_vocab}
    words_not_in_vocab = {word for word in tokenized_words if word not in bart_vocab}

    return words_in_vocab, words_not_in_vocab

In [ ]:
words_in_vocab, words_not_in_vocab = compare_words_with_bart_vocab(unique_words)

print("Words in BART vocabulary:", words_in_vocab)
print("Words not in BART vocabulary:", words_not_in_vocab)

Words in BART vocabulary: {'sche', 'process', 'will', 'me', 'acts', 'ly', 'enced', 'credit', 'ulated', 'co', 'turned', 'ifted', 'man', 'portation', 'ra', 'feet', 'hold', 'else', 'defense', 'aste', 'ot', 'over', 'opa', 'song', 'offic', 'strong', 'done', 'foot', 'ophy', 'Ä«', 'oteric', 'd', 'att', 'period', 'pl', 'intention', 'mess', 'ration', 'rig', 'handle', 'ressing', 'res', 'iences', 'powerful', 'travel', 'cat', 'anners', 'performance', 'whether', 'gr', 'pad', 'ourage', 'ulsion', 'uring', 'lu', 'ing', 'avored', 'scape', 'asons', 'iveness', 'work', 'object', 'rival', 'isk', '£', 'icular', 'reci', 'humans', 'rings', 'ores', 'gro', 'keeping', 'akin', 'iam', 'dule', 'rates', 'iblings', 'centered', 'instance', 'aled', 'strate', 'rel', 'shield', 'protect', 'pan', 'given', 'lies', 'ola', 'struct', 'vari', 'cond', 'age', 'occup', 'orts', 'osing', 'chall', 'events', 'aggressive', 'ard', 'uations', 'ag', 'engineering', 'second', 'inction', 'bowl', 'kid', 'el', 'eatured', 'acement', 'ask', 'add

In [ ]:
print(f"Words to be added: {len(words_not_in_vocab)}")

Words to be added: 0


In [ ]:
def add_new_tokens_to_vocab(new_tokens):
    num_added_tokens = tokenizer.add_tokens(new_tokens)
    model.resize_token_embeddings(len(tokenizer))
    vocab = tokenizer.get_vocab()
    print("Vocabulary size:", len(vocab))

new_tokens = list(words_not_in_vocab)
num_added_tokens = add_new_tokens_to_vocab(new_tokens)

Vocabulary size: 50265


In [67]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import wandb

# Define the function to find the best match for the answer in the context
from difflib import SequenceMatcher

def find_best_match(answer, context):
    match = SequenceMatcher(None, context, answer).find_longest_match(0, len(context), 0, len(answer))
    return match.a, match.a + match.size - 1 if match.size > 0 else (-1, -1)

# QADataset Class
class QADataset(Dataset):
    """Dataset for Question Answering with Context and Answer Alignment"""
    def __init__(self, data, tokenizer, max_input_length=512, max_output_length=128):
        """
        Args:
            data (pd.DataFrame): Dataset with 'Question', 'Object', 'Answer', and 'Context'.
            tokenizer (transformers.PreTrainedTokenizer): Tokenizer to encode inputs.
            max_input_length (int): Maximum length for tokenized context+question.
            max_output_length (int): Maximum length for tokenized answers.
        """
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract relevant fields from the dataset
        question = self.data.iloc[idx]['Question']
        object_name = self.data.iloc[idx]['Object']
        answer = self.data.iloc[idx]['Answer']
        context = self.data.iloc[idx]['Context']

        # Combine the object name (context) and question
        combined = object_name + " " + question + " " + context

        # Tokenize the combined input (question + context) with padding to the longest sequence in the batch
        inputs = self.tokenizer(
            combined,
            padding='longest',  # Pad to the longest sequence in the batch
            truncation=True,
            max_length=self.max_input_length,
            return_tensors="pt"
        )

        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()

        # Tokenize the answer to locate it within the context
        answer_tokens = self.tokenizer(
            answer,
            padding='do_not_pad',  # No padding for token matching
            truncation=True,
            max_length=self.max_output_length,
            return_tensors="pt"
        ).input_ids.squeeze()

        # Find the start and end positions of the answer in the input_ids
        answer_length = len(answer_tokens)
        start_positions, end_positions = -1, -1
        for start_idx in range(len(input_ids) - answer_length + 1):
            if torch.equal(input_ids[start_idx:start_idx + answer_length], answer_tokens):
                start_positions = start_idx
                end_positions = start_idx + answer_length - 1
                break

        # Handle cases where the answer is not found using best match
        if start_positions == -1 or end_positions == -1:
            start_positions, end_positions = find_best_match(answer, context)

        # Return the tokenized input and answer positions
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': start_positions,
            'end_positions': end_positions
        }

# Collate function for batching
def collate_fn(batch):
    # Use padding to ensure each tensor has the same size
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    start_positions = [item['start_positions'] for item in batch]
    end_positions = [item['end_positions'] for item in batch]

    # Pad the sequences
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    start_positions = torch.tensor(start_positions)
    end_positions = torch.tensor(end_positions)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'start_positions': start_positions,
        'end_positions': end_positions
    }

In [68]:
model

BartForQuestionAnswering(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [69]:
train_dataset = QADataset(train_df, tokenizer=tokenizer)
val_dataset = QADataset(val_df, tokenizer=tokenizer)
test_dataset = QADataset(test_df, tokenizer=tokenizer)

In [70]:
import torch
from transformers import AdamW
import wandb
from tqdm import tqdm
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer


class VQATrainer:
    """Class for Trainer Setup to Train the BART Model for Question Answering"""

    def __init__(self, model, train_dataloader, eval_dataloader, device, config):
        """Constructor"""
        self.model = model
        self.train_dataloader = train_dataloader
        self.eval_dataloader = eval_dataloader
        self.device = device
        self.optimizer = AdamW(self.model.parameters())

        # Initialize wandb for tracking experiments
        wandb.init(project=config['project_name'], config=config)
        wandb.watch(self.model, log="all")

    def evaluate(self):
        """Evaluate the model"""
        self.model.eval()
        total_loss = 0
        all_predictions = []
        all_references = []

        progress_bar = tqdm(self.eval_dataloader, desc="Evaluating")

        for batch in progress_bar:
            with torch.no_grad():
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                start_positions = batch['start_positions'].to(self.device)
                end_positions = batch['end_positions'].to(self.device)

                # Forward pass for evaluation
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    start_positions=start_positions,
                    end_positions=end_positions
                )

                loss = outputs.loss
                total_loss += loss.item()

                # Predicted start and end positions
                start_preds = torch.argmax(outputs.start_logits, dim=1).cpu().tolist()
                end_preds = torch.argmax(outputs.end_logits, dim=1).cpu().tolist()

                # Convert token IDs to text
                predictions = [
                    self.model.tokenizer.decode(input_ids[i][start_preds[i]:end_preds[i] + 1])
                    for i in range(len(start_preds))
                ]
                references = [
                    self.model.tokenizer.decode(input_ids[i][start_positions[i]:end_positions[i] + 1])
                    for i in range(len(start_positions))
                ]

                all_predictions.extend(predictions)
                all_references.extend(references)

        avg_loss = total_loss / len(self.eval_dataloader)

        # Token-level accuracy
        token_accuracy = np.mean([
            1 if pred == ref else 0
            for pred, ref in zip(all_predictions, all_references)
        ])

        # Calculate BLEU score
        smoothing_fn = SmoothingFunction().method1
        bleu_scores = [
            sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothing_fn)
            for pred, ref in zip(all_predictions, all_references)
        ]
        avg_bleu = np.mean(bleu_scores)

        # Calculate ROUGE scores
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(all_references, all_predictions)]
        avg_rouge = {
            key: np.mean([score[key].fmeasure for score in rouge_scores])
            for key in ['rouge1', 'rouge2', 'rougeL']
        }

        # Log metrics
        metrics = {
            "Validation Loss": avg_loss,
            "Token-Level Accuracy": token_accuracy,
            "BLEU Score": avg_bleu,
            "ROUGE-1 F1": avg_rouge['rouge1'],
            "ROUGE-2 F1": avg_rouge['rouge2'],
            "ROUGE-L F1": avg_rouge['rougeL']
        }

        print(f"Evaluation Metrics: {metrics}")
        wandb.log(metrics)
        return metrics

    def train_epoch(self):
        """Train for a single epoch"""
        self.model.train()
        total_loss = 0
        progress_bar = tqdm(self.train_dataloader, desc="Training")

        for batch in progress_bar:
            self.optimizer.zero_grad()

            # Move inputs and labels to the device
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            start_positions = batch['start_positions'].to(self.device)
            end_positions = batch['end_positions'].to(self.device)

            # Forward pass
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions
            )
            loss = outputs.loss
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
            wandb.log({"train_loss": loss.item()})

        avg_loss = total_loss / len(self.train_dataloader)
        print(f"Average Training Loss: {avg_loss}")

    def train(self, epochs):
        """Train the model for a number of epochs"""
        for epoch in range(epochs):
            print(f'Epoch {epoch + 1}/{epochs}')
            self.train_epoch()
            metrics = self.evaluate()
            print(f"Metrics: {metrics}")
            torch.save(self.model.state_dict(), f"model_epoch_{epoch}.pth")


In [71]:
api_key = os.getenv('API_KEY')
!wandb login --relogin $api_key

Traceback (most recent call last):
  File "/usr/local/bin/wandb", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/cli/cli.py", line 104, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/cli/cli.py", line 254, in login
    wandb.login(
  File "/usr/local/lib/python3.1

In [72]:
config = {
     "batch_size":8,
     "epochs" : 10,
     "model_name": "valhalla/bart-large-finetuned-squadv1",
     "project_name": "VQA_BART_valhalla",
}

In [73]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)

In [74]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForQuestionAnswering.from_pretrained(config['model_name']).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [75]:
trainer = VQATrainer(model, train_dataloader, val_dataloader,device,config)

In [76]:
trainer.train(config['epochs'])

Epoch 1/10


Training:   0%|          | 0/1078 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [124] at entry 0 and [132] at entry 1

In [ ]:
wandb.finish()